In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


#### After import necessary package, using BeautifulSoup to obtain the top 100 cities

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
wiki_source = requests.get(wiki_url).text
soup = BeautifulSoup(wiki_source, 'lxml')
top100table = soup.find_all('tbody')[4].find_all('td')
for i in range(len(top100table)):
    top100table[i] = top100table[i].text

#### Create a DataFrame to store the top 100 cities.

In [3]:
city_data = pd.DataFrame(columns = ['City', 'Latitude', 'Longitude'])

#### Find their city name, latitude and longitude

In [4]:
numList = []
for i in range(10):
    numList.append(str(i))
cityL = []
latitudeL = []
longitudeL = []
for i in range(100):
    tmpstr = top100table[11*i+10].strip()[46:]
    pos1 = tmpstr.find('/')
    pos2 = tmpstr.find(';')
    pos3 = tmpstr.find('(')
    pos4 = tmpstr.find(')')
    latitudeL.append(float(tmpstr[pos1+2:pos2]))
    longitudeL.append(float(tmpstr[pos2+2:pos3-2]))
    citytmp = tmpstr[pos3+1:pos4]
    if citytmp[1] in numList:
        citytmp = citytmp[3:]
    elif citytmp[0] in numList:
        citytmp = citytmp[2:]
    cityL.append(citytmp)
city_data['City'] = cityL
city_data['Latitude'] = latitudeL
city_data['Longitude'] = longitudeL

#### Let's see how the data of top 100 cities looks like

In [5]:
city_data.head()

,City,Latitude,Longitude
0,New York City,40.6635,-73.9387
1,Los Angeles,34.0194,-118.4108
2,Chicago,41.8376,-87.6818
3,Houston,29.7866,-95.3909
4,Phoenix,33.5722,-112.0901


#### For each city, find venues categories, latitude and longitude in the city

In [6]:
CLIENT_ID = 'WKKBTHRYCYXWIQKBCQDP34LJC2CSKBUKDUAWH5YYSUZNOAB5' # your Foursquare ID
CLIENT_SECRET = '5WSUFFDLI1CDS4RM24S0TRQQMCRI3151JR2JWIT0JQZXC3FP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 999
names = city_data['City']
latitudes = city_data['Latitude']
longitudes = city_data['Longitude']
venues_list=[]
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['City', 
                'City Latitude', 
                'City Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']

#### Let's see how the data of venues in the top 100 cities looks like

In [7]:
nearby_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York City,40.6635,-73.9387,Izzy's Brooklyn Smokehouse,40.664869,-73.937023,BBQ Joint
1,New York City,40.6635,-73.9387,Ali's Roti Shop,40.666436,-73.931346,Caribbean Restaurant
2,New York City,40.6635,-73.9387,Sweet Expressions,40.668382,-73.942337,Candy Store
3,New York City,40.6635,-73.9387,Bodega,40.668388,-73.932366,Deli / Bodega
4,New York City,40.6635,-73.9387,The Market Place,40.662153,-73.943010,Grocery Store


#### Use get_dummies function to transform the table

In [8]:
city_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
city_onehot['City'] = nearby_venues['City'] 
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

#### Let's see hot the data looks like

In [9]:
city_onehot.head()

,City,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Bookstore,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / B

#### Helper function to find the most common venues

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

#### Find the 10 most common venues in each city

In [11]:
city_grouped = city_onehot.groupby('City').mean().reset_index()
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = city_grouped['City']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

#### Let's see the 10 most common categories of venues categories in each city

In [12]:
city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albuquerque,Mexican Restaurant,Brewery,Construction & Landscaping,Discount Store,Intersection,Event Service,Lawyer,Gas Station,Sandwich Place,Furniture / Home Store
1,Anaheim,Japanese Restaurant,Dance Studio,Home Service,Thai Restaurant,Burger Joint,Sushi Restaurant,Mediterranean Restaurant,Sandwich Place,Gym / Fitness Center,Fast Food Restaurant
2,Arlington,Mexican Restaurant,Breakfast Spot,Rental Car Location,Donut Shop,Burger Joint,Gym,Smoke Shop,Martial Arts Dojo,Food Stand,Bowling Alley
3,Atlanta,Convenience Store,Park,Gay Bar,Pool,Wings Joint,Light Rail Station,Food,Tennis Court,Gas Station,Nightclub
4,Austin,Park,Coffee Shop,Sandwich Place,American Restaurant,Rental Car Location,Men's Store,Gift Shop,Italian Restaurant,Grocery Store,Pharmacy


#### Helper function to show map

In [13]:
def createMap(kclusters, city_merge):
    map_clusters = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    markers_colors = []
    for lat, lon, poi, cluster in zip(city_merge['Latitude'], city_merge['Longitude'], city_merge['City'], city_merge['Cluster Labels']):
        if cluster not in [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]:
            continue
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    return map_clusters

#### Let's try to cluster the cities in 2 clusters based on their venues' categories

In [26]:
kclusters = 2
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
city_merge.head()

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York City,40.6635,-73.9387,0.0,Pizza Place,Grocery Store,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Pharmacy,Donut Shop,Fast Food Restaurant,Bagel Shop,Discount Store
1,Los Angeles,34.0194,-118.4108,0.0,Indian Restaurant,Convenience Store,Taco Place,Bakery,Thai Restaurant,Sushi Restaurant,Coffee Shop,Sandwich Place,Mexican Restaurant,Burger Joint
2,Chicago,41.8376,-87.6818,0.0,Grocery Store,Diner,Coffee Shop,Fast Food Restaurant,Public Art,Bus Stop,Bakery,Taco Place,Furniture / Home Store,Big Box Store
3,Houston,29.7866,-95.3909,0.0,Mexican Restaurant,Coffee Shop,Café,Ice Cream Shop,Food Truck,Gym / Fitness Center,American Restaurant,Bar,Park,Italian Restaurant
4,Phoenix,33.5722,-112.0901,0.0,Fast Food Restaurant,Food Truck,Sandwich Place,Nail Salon,Bar,Video Store,Pharmacy,Bookstore,Animal Shelter,Mexican Restaurant


#### Show the Map

In [27]:
map = createMap(2, city_merge)
map

#### Let's cluster the cities in 3 clusters

In [16]:
kclusters = 3
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 4 clusters

In [17]:
kclusters = 4
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 5 clusters

In [18]:
kclusters = 5
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 6 clusters

In [19]:
kclusters = 6
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 7 clusters

In [20]:
kclusters = 7
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 8 clusters

In [21]:
kclusters = 8
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 9 clusters

In [22]:
kclusters = 9
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

#### Let's cluster the cities in 10 clusters

In [32]:
kclusters = 10
city_grouped_clustering = city_grouped.drop('City', axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
city_venues_sorted_copy = city_venues_sorted.copy()
city_venues_sorted_copy.insert(0, 'Cluster Labels', kmeans.labels_)
city_merge = city_data
city_merge = city_merge.join(city_venues_sorted_copy.set_index('City'), on='City')
map = createMap(kclusters, city_merge)
map

### Le's see the 10 clusters

#### Cluster 1

In [33]:
city_merge.loc[city_merge['Cluster Labels'] == 0.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Denver,Recreation Center,Park,Pool,Restaurant,Zoo Exhibit,Farm,Event Service,Event Space,Exhibit,Eye Doctor
20,Boston,Park,Beach,Baseball Field,Hot Dog Joint,Lighthouse,History Museum,Historic Site,American Restaurant,Trail,Playground
27,Las Vegas,Park,Trail,Fast Food Restaurant,Skate Park,Convenience Store,Taco Place,Farm,Event Space,Exhibit,Eye Doctor
37,Atlanta,Convenience Store,Park,Gay Bar,Pool,Wings Joint,Light Rail Station,Food,Tennis Court,Gas Station,Nightclub
55,Honolulu,Park,Event Service,Sushi Restaurant,Martial Arts Dojo,Bus Stop,Cafeteria,Theater,Chinese Restaurant,Astrologer,Vegetarian / Vegan Restaurant
75,Chula Vista,Park,Outdoor Gym,Dance Studio,Intersection,Fast Food Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory
92,Irving,Baseball Field,Ice Cream Shop,Home Service,Convenience Store,Park,Donut Shop,Athletics & Sports,Sandwich Place,Farmers Market,Exhibit
96,Richmond,Park,Beach,Trail,Garden,Forest,Bridge,Farm,Museum,Lake,Garden Center


#### Cluster 2

In [34]:
city_merge.loc[city_merge['Cluster Labels'] == 1.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York City,Pizza Place,Grocery Store,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Pharmacy,Donut Shop,Fast Food Restaurant,Bagel Shop,Discount Store
1,Los Angeles,Indian Restaurant,Convenience Store,Taco Place,Bakery,Thai Restaurant,Sushi Restaurant,Coffee Shop,Sandwich Place,Mexican Restaurant,Burger Joint
2,Chicago,Grocery Store,Diner,Coffee Shop,Fast Food Restaurant,Public Art,Bus Stop,Bakery,Taco Place,Furniture / Home Store,Big Box Store
3,Houston,Mexican Restaurant,Coffee Shop,Café,Ice Cream Shop,Food Truck,Gym / Fitness Center,American Restaurant,Bar,Park,Italian Restaurant
9,San Jose,Fast Food Restaurant,Playground,Fried Chicken Joint,Golf Course,Sandwich Place,Bubble Tea Shop,Caribbean Restaurant,Shipping Store,Electronics Store,Coffee Shop
10,Austin,Park,Coffee Shop,Sandwich Place,American Restaurant,Rental Car Location,Men's Store,Gift Shop,Italian Restaurant,Grocery Store,Pharmacy
11,Jacksonville,Sandwich Place,Park,Deli / Bodega,Art Gallery,Coffee Shop,Convenience Store,Pub,Restaurant,Storage Facility,Speakeasy
14,Fort Worth,Bar,Mexican Restaurant,Steakhouse,Hotel,Historic Site,Burger Joint,Tourist Information Center,Theater,Herbs & Spices Store,New American Restaurant
15,Indianapolis,Bar,Brewery,Pub,American Restaurant,Gastropub,Coffee Shop,Theater,New American Restaurant,Wine Bar,Comedy Club
16,Charlotte,Pizza Place,Coffee Shop,Ice Cream Shop,American Restaurant,Fast Food Restaurant,Bakery,Sandwich Place,Bed & Breakfast,Italian Restaurant,Convenience Store


#### Cluster 3

In [35]:
city_merge.loc[city_merge['Cluster Labels'] == 2.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Orlando,Movie Theater,Gym,Lake,Furniture / Home Store,Zoo Exhibit,Fast Food Restaurant,Exhibit,Eye Doctor,Fabric Shop,Factory


#### Cluster 4

In [36]:
city_merge.loc[city_merge['Cluster Labels'] == 3.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Henderson,Auto Workshop,Golf Course,Event Service,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market


#### Cluster 5

In [37]:
city_merge.loc[city_merge['Cluster Labels'] == 4.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Phoenix,Fast Food Restaurant,Food Truck,Sandwich Place,Nail Salon,Bar,Video Store,Pharmacy,Bookstore,Animal Shelter,Mexican Restaurant
5,Philadelphia,Donut Shop,Intersection,Gym,Fast Food Restaurant,Supermarket,Gas Station,BBQ Joint,Food,Fried Chicken Joint,Pharmacy
6,San Antonio,Mexican Restaurant,Pharmacy,Grocery Store,Convenience Store,Taco Place,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Food Truck,Sporting Goods Shop
7,San Diego,Mexican Restaurant,Café,Sandwich Place,Thai Restaurant,American Restaurant,Arts & Crafts Store,Argentinian Restaurant,Donut Shop,Dry Cleaner,Airport Terminal
8,Dallas,Mexican Restaurant,Pizza Place,Convenience Store,Donut Shop,Sandwich Place,Bank,Taco Place,Nightclub,New American Restaurant,Chinese Restaurant
21,El Paso,Shoe Store,Mexican Restaurant,Bakery,Baseball Field,Japanese Restaurant,Food,Farmers Market,Food Court,Eye Doctor,Fabric Shop
24,Memphis,Fast Food Restaurant,Convenience Store,Discount Store,Shop & Service,Plaza,Intersection,Electronics Store,BBQ Joint,Sandwich Place,Grocery Store
31,Albuquerque,Mexican Restaurant,Brewery,Construction & Landscaping,Discount Store,Intersection,Event Service,Lawyer,Gas Station,Sandwich Place,Furniture / Home Store
33,Fresno,Mexican Restaurant,Chinese Restaurant,Fast Food Restaurant,Hotel,Liquor Store,Coffee Shop,Food Truck,Buffet,Nightclub,Shopping Mall
35,Mesa,Sandwich Place,Massage Studio,Dance Studio,Eastern European Restaurant,American Restaurant,Discount Store,Golf Course,Martial Arts Dojo,Field,Eye Doctor


#### Cluster 6

In [38]:
city_merge.loc[city_merge['Cluster Labels'] == 5.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Columbus,Fried Chicken Joint,Fast Food Restaurant,Intersection,Zoo Exhibit,Field,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant


#### Cluster 7

In [39]:
city_merge.loc[city_merge['Cluster Labels'] == 6.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,Madison,Trail,Stadium,Baseball Field,Garden,Fast Food Restaurant,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant


#### Cluster 8

In [40]:
city_merge.loc[city_merge['Cluster Labels'] == 7.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Detroit,Fast Food Restaurant,Neighborhood,Optical Shop,Park,Construction & Landscaping,Liquor Store,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
32,Tucson,Construction & Landscaping,Fast Food Restaurant,Business Service,Zoo Exhibit,Field,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
68,Plano,Fast Food Restaurant,Park,Convenience Store,American Restaurant,Pizza Place,Carpet Store,Business Service,Zoo Exhibit,Exhibit,Eye Doctor


#### Cluster 9

In [41]:
city_merge.loc[city_merge['Cluster Labels'] == 8.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,Scottsdale,Beach,Zoo Exhibit,Event Space,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


#### Cluster 10

In [42]:
city_merge.loc[city_merge['Cluster Labels'] == 9.0, city_merge.columns[[0] + list(range(4, city_merge.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,Laredo,Hotel,Coffee Shop,Dessert Shop,Snack Place,Café,Zoo Exhibit,Fast Food Restaurant,Eye Doctor,Fabric Shop,Factory
